In [485]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [486]:
#importing files
ind_master = pd.read_excel('Ind_Residents_Merged.xlsx')
sample = pd.read_excel('Data - Individual Samples MNM.v3.xlsx')

#cleaning sample df to prepare for merge
sample = sample.drop(labels=['result_value', 'result','characteristic_name'], axis=1)
sample = sample.melt(id_vars=['first_name','last_name','address','city','state','zip_code','site_id','source','program','collected_by',
    'collection_point','collection_date','result_value_unit','treatment', 'notes'],
     var_name=['characteristic_name'], value_name='result_value')

#cleaning ind_master df to prepare for merge
ind_master = ind_master.drop(labels =['analyzed_datetime','new_datetime_format'], axis=1)
ind_master = ind_master.rename(columns={'result_value_units':'result_value_unit'})

#merging two dfs
final = pd.concat([ind_master, sample1])
final.head()

,first_name,last_name,address,city,state,zip_code,site_id,map_results,source,program,...,reporting_limit,analytical_method_id,lab_sample_id,batch_id,treatment,depth_well,lat,long,notes,collection_point
0,Dundee,Warden,1820 Arrastra Gulch Dr,Helena,MT,59601.0,NaN,YES,ENERGY LAB,WELL EDUCATED,...,4,NaN,B09040494‐001,NaN,UNKNOWN TREATMENT,NaN,46.548877,-112.062734,NaN,NaN
1,Dundee,Warden,1820 Arrastra Gulch Dr,Helena,MT,59601.0,NaN,YES,ENERGY LAB,WELL EDUCATED,...,0.1,NaN,B09040494‐001,NaN,UNKNOWN TREATMENT,NaN,46.548877,-112.062734,NaN,NaN
2,Dundee,Warden,1820 Arrastra Gulch Dr,Helena,MT,59601.0,NaN,YES,ENERGY LAB,WELL EDUCATED,...,0.001,E200.8,B09040494‐001,NaN,UNKNOWN TREATMENT,NaN,46.548877,-112.062734,NaN,NaN
3,Dundee,Warden,1820 Arrastra Gulch Dr,Helena,MT,59601.0,NaN,YES,ENERGY LAB,WELL EDUCATED,...,4,NaN,B09040494‐001,NaN,UNKNOWN TREATMENT,NaN,46.548877,-112.062734,NaN,NaN
4,Dundee,Warden,1820 Arrastra Gulch Dr,Helena,MT,59601.0,NaN,YES,ENERGY LAB,WELL EDUCATED,...,0.001,NaN,B09040494‐001,NaN,UNKNOWN TREATMENT,NaN,46.548877,-112.062734,NaN,NaN


In [487]:
#cleaning final df
final = final.apply(lambda x: x.astype(str).str.upper())
final['collection_date'] = pd.to_datetime(final['collection_date']).dt.strftime('%m/%d/%Y')
final = final.apply(lambda x: x.str.strip('""'))
final = final.rename(columns={'reporting_limit':'lower_reporting_limit'})

In [488]:
#replacing the values and removing elements with '<' AND '<' character
final['result'] = np.where(final['result_value'].str.contains('<'), 'ND', final['result'])
final['lower_reporting_limit'] = np.where(final['result_value'].str.contains('<'), final['result_value'], final['lower_reporting_limit'])
final['result_value'] = np.where(final['result_value'].str.contains('<'), '', final['result_value'])
final['lower_reporting_limit'] = final['lower_reporting_limit'].str.strip('<')

In [489]:
#removing null values for BOTH the result_value and result columns
final = final.drop(final[(final['result_value'] == 'NAN') & (final['result'] == 'NAN')].index)

In [490]:
#switching elements for result and result_values columns
final['result'] = np.where(final['result_value'] == 'ABSENT','ABSENT', final['result'])
final['result_value'] = np.where(final['result_value'] == 'ABSENT','', final['result_value'])

final['result'] = np.where(final['result_value'] == 'PRESENT','PRESENT', final['result'])
final['result_value'] = np.where(final['result_value'] == 'PRESENT','', final['result_value'])

final['result'] = np.where(final['result_value'] == 'ND','ND', final['result'])
final['result_value'] = np.where(final['result_value'] == 'ND', '', final['result_value'])

final['result_value'] = np.where(final['result_value'] == 'NAN', '', final['result_value'])
final['result'] = np.where(final['result'] == 'NAN','', final['result'])

final.head()

,first_name,last_name,address,city,state,zip_code,site_id,map_results,source,program,...,lower_reporting_limit,analytical_method_id,lab_sample_id,batch_id,treatment,depth_well,lat,long,notes,collection_point
1,DUNDEE,WARDEN,1820 ARRASTRA GULCH DR,HELENA,MT,59601.0,NAN,YES,ENERGY LAB,WELL EDUCATED,...,0.1,NAN,B09040494‐001,NAN,UNKNOWN TREATMENT,NAN,46.548877,-112.062734,NAN,NAN
3,DUNDEE,WARDEN,1820 ARRASTRA GULCH DR,HELENA,MT,59601.0,NAN,YES,ENERGY LAB,WELL EDUCATED,...,4,NAN,B09040494‐001,NAN,UNKNOWN TREATMENT,NAN,46.548877,-112.062734,NAN,NAN
5,DUNDEE,WARDEN,1820 ARRASTRA GULCH DR,HELENA,MT,59601.0,NAN,YES,ENERGY LAB,WELL EDUCATED,...,1,NAN,B09040494‐001,NAN,UNKNOWN TREATMENT,NAN,46.548877,-112.062734,NAN,NAN
6,DUNDEE,WARDEN,1820 ARRASTRA GULCH DR,HELENA,MT,59601.0,NAN,YES,ENERGY LAB,WELL EDUCATED,...,4,NAN,B09040494‐001,NAN,UNKNOWN TREATMENT,NAN,46.548877,-112.062734,NAN,NAN
8,DUNDEE,WARDEN,1820 ARRASTRA GULCH DR,HELENA,MT,59601.0,NAN,YES,ENERGY LAB,WELL EDUCATED,...,NAN,A9223 B,B09040494‐001,NAN,UNKNOWN TREATMENT,NAN,46.548877,-112.062734,NAN,NAN


In [491]:
#consolidating characteristic_name values
final['characteristic_name'].replace(['NITROGEN, NITRATE+NITRITE AS N', 'NITRATE PLUS NITRITE AS N'], 'NITRATE PLUS NITRITE AS N', inplace=True)
final['characteristic_name'].replace(['SPECIFIC CONDUCTANCE'], 'SPECIFIC CONDUCTIVITY', inplace=True) 
final['characteristic_name'].replace(['BICARBONATE AS HCO3'],'BICARBONATE, AS HCO3', inplace=True)
final['characteristic_name'].replace(['E-COLI'],'COLIFORM, E‐COLI', inplace=True)
final['characteristic_name'].replace(['TOTAL COLIFORM'],'COLIFORM, TOTAL', inplace=True)
final['characteristic_name'].replace(['TOTAL HARDNESS AS CACO3'],'HARDNESS, TOTAL AS CACO3', inplace=True)

In [492]:
#changing result value column to appropriate units
char = ['TOTAL DISSOLVED SOLIDS', 'ALKALINITY, TOTAL AS CACO3', 'BICARBONATE, AS CACO3',
       'CHLORIDE', 'SULFATE', 'FLUORIDE', 'HARDNESS, TOTAL AS CACO3', 'NITRATE PLUS NITRITE AS N', 'ARSENIC',
      'ALUMINUM', 'CALCIUM', 'CADMIUM', 'CHROMIUM', 'COPPER', 'IRON', 'LEAD', 'MAGNESIUM', 'MANGANESE', 'POTASSIUM',
       'SELENIUM', 'SODIUM', 'URANIUM', 'ZINC', 'THORIUM', 'BICARBONATE, AS HCO3']

for i in char:
    final['result_value_unit'] = np.where(final['characteristic_name'] == i, 'MG/L',
                                      final['result_value_unit'])

char2 = 'SPECIFIC CONDUCTIVITY'
final['result_value_unit'] =  np.where(final['characteristic_name'] == char2, 'UMHO/CM',final['result_value_unit'])
                                       
char3 = 'COLIFORM, TOTAL'                                      
final['result_value_unit'] =  np.where(final['characteristic_name'] == char3, 'P/A',final['result_value_unit'])
                                      
char4 = 'COLIFORM, E‐COLI' 
final['result_value_unit'] =  np.where(final['characteristic_name'] == char4, 'P/A', final['result_value_unit'])

In [495]:
#reformatting address
add = final[final['state'] == 'NAN']
final = final.loc[final['state'] != 'NAN']
add[['address', 'city', 'state', 'zip_code']] = add['address'].str.rsplit(',', n=3, expand=True)
final = pd.concat([final, add])
final

,first_name,last_name,address,city,state,zip_code,site_id,map_results,source,program,...,lower_reporting_limit,analytical_method_id,lab_sample_id,batch_id,treatment,depth_well,lat,long,notes,collection_point
1,DUNDEE,WARDEN,1820 ARRASTRA GULCH DR,HELENA,MT,59601.0,NAN,YES,ENERGY LAB,WELL EDUCATED,...,0.1,NAN,B09040494‐001,NAN,UNKNOWN TREATMENT,NAN,46.548877,-112.062734,NAN,NAN
3,DUNDEE,WARDEN,1820 ARRASTRA GULCH DR,HELENA,MT,59601.0,NAN,YES,ENERGY LAB,WELL EDUCATED,...,4,NAN,B09040494‐001,NAN,UNKNOWN TREATMENT,NAN,46.548877,-112.062734,NAN,NAN
5,DUNDEE,WARDEN,1820 ARRASTRA GULCH DR,HELENA,MT,59601.0,NAN,YES,ENERGY LAB,WELL EDUCATED,...,1,NAN,B09040494‐001,NAN,UNKNOWN TREATMENT,NAN,46.548877,-112.062734,NAN,NAN
6,DUNDEE,WARDEN,1820 ARRASTRA GULCH DR,HELENA,MT,59601.0,NAN,YES,ENERGY LAB,WELL EDUCATED,...,4,NAN,B09040494‐001,NAN,UNKNOWN TREATMENT,NAN,46.548877,-112.062734,NAN,NAN
8,DUNDEE,WARDEN,1820 ARRASTRA GULCH DR,HELENA,MT,59601.0,NAN,YES,ENERGY LAB,WELL EDUCATED,...,NAN,A9223 B,B09040494‐001,NAN,UNKNOWN TREATMENT,NAN,46.548877,-112.062734,NAN,NAN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24013,BRAD,BORST,8157 RED RIDGE DR,HELENA,MT,59602,NAN,NAN,NAN,RESIDENTIAL SAMPLING,...,NAN,NAN,NAN,NAN,UNKNOWN TREATMENT,NAN,NAN,NAN,NAN,NAN
24019,PHILLIP,CHAMBLEY,6289 TIMBER TRAIL DR,HELENA,MT,59602,NAN,NAN,NAN,RESIDENTIAL SAMPLING,...,NAN,NAN,NAN,NAN,UNKNOWN TREATMENT,NAN,NAN,NAN,NAN,NAN
24020,JON,ALBERTSON,200 JOHN G MINE RD,HELENA,MT,59602,NAN,NAN,NAN,RESIDENTIAL SAMPLING,...,NAN,NAN,NAN,NAN,UNKNOWN TREATMENT,NAN,NAN,NAN,NAN,NAN
24021,GARY,SWANSON,227 PARSLEY RD,HELENA,MT,59602,NAN,NAN,NAN,RESIDENTIAL SAMPLING,...,NAN,NAN,NAN,NAN,UNKNOWN TREATMENT,NAN,NAN,NAN,NAN,NAN
